In [1]:
import numpy as np
import pandas as pd 
import pickle

In [2]:
multivariate_datasets = [
    "Cricket", 
    "ERing", 
    "Handwriting",
    "Libras",
    "NATOPS",
    "RacketSports",
    "UWaveGestureLibrary",
    "ArticularyWordRecognition", 
]

univariate_datasets = [
    "ArrowHead", 
    "BME", 
    "ECG200",
    "FacesUCR",
    "GunPoint", 
    "PhalangesOutlinesCorrect",
    "Trace",
]

valid_datasets = univariate_datasets + multivariate_datasets

lst = [[ds,"univariate"] for ds in univariate_datasets] + [[ds, "multivariate"] for ds in multivariate_datasets]
ds_df = pd.DataFrame(lst, columns= ["dataset","type"])

# Shape analysis classification

In [3]:
fpca_df = pd.read_csv("results/shape-fpca_0.0_0.csv",index_col=0)
fpca_df["model"] = "Shape-FPCA (2024)"
tclr_df = pd.read_csv("results/tclr_0.0_0.csv",index_col=0)
tclr_df["model"] = "TCLR (2024)"
lddmm_df = pd.read_csv("results/lddmm_methods_0.0_0.csv",index_col=0)
lddmm_df["model"] = lddmm_df["model"].apply(lambda x : "TS-LDDMM (ours)" if x == "ts-lddmm" else "LDDMM (2008)")
df = pd.concat([fpca_df,tclr_df,lddmm_df]).reset_index(drop = True)
df = pd.merge(df,ds_df,on="dataset")
df = pd.pivot_table(df,"fscore",["type","dataset"],"model",aggfunc="first")
df = df.loc[["univariate","multivariate"]]
df = df[["Shape-FPCA (2024)", "TCLR (2024)", "LDDMM (2008)","TS-LDDMM (ours)"]]
df_s = df.style.format("{:.2f}")

# loop through rows and find which column for each row has the highest value
for row in df.index:
    col = df.loc[row].idxmax()
    col2 = df.loc[row].nlargest(2).idxmin()
    # redo formatting for a specific cell
    df_s = df_s.format(lambda x: "\\textbf{" + f'{x:.2f}' + "}", subset=(row, col))
    df_s = df_s.format(lambda x: "\\underline{" + f'{x:.2f}' + "}", subset=(row, col2))

row,col = np.where(df.isna())
row = df.index[row]
col = df.columns[col]
df_s.format(lambda x : "--",subset=(row,col))


print(df_s.to_latex(hrules = True, clines = "skip-last;data",multirow_align = "t"))


\begin{tabular}{llrrrr}
\toprule
 & model & Shape-FPCA (2024) & TCLR (2024) & LDDMM (2008) & TS-LDDMM (ours) \\
type & dataset &  &  &  &  \\
\midrule
\multirow[t]{7}{*}{univariate} & ArrowHead & 0.18 & \textbf{29.72} & 0.84 & \underline{0.91} \\
 & BME & 0.16 & \textbf{8.02} & 0.82 & \underline{1.00} \\
 & ECG200 & 0.40 & \textbf{5.89} & \underline{0.81} & 0.79 \\
 & FacesUCR & 0.08 & \textbf{77.90} & 0.69 & \underline{0.86} \\
 & GunPoint & 0.93 & \textbf{11.42} & 0.83 & \underline{1.00} \\
 & PhalangesOutlinesCorrect & 0.39 & \textbf{40.95} & \underline{0.53} & 0.52 \\
 & Trace & 0.55 & \textbf{33.96} & 0.46 & \underline{1.00} \\
\cline{1-6}
\multirow[t]{8}{*}{multivariate} & ArticularyWordRecognition & -- & -- & \underline{0.98} & \textbf{1.00} \\
 & Cricket & -- & -- & \underline{0.77} & \textbf{0.93} \\
 & ERing & -- & -- & \underline{0.95} & \textbf{0.98} \\
 & Handwriting & -- & -- & \underline{0.22} & \textbf{0.44} \\
 & Libras & -- & -- & \underline{0.56} & \textbf{0.60} \\
 

# Robustness

In [4]:
model_mapping = dict(
    rnn = "RNN (1999)",
    lstm = 'LSTM (1997)',
    gru = "GRU (2014)",
    mtan = 'MTAN (2021)',
    miam = 'MIAM (2022)',
    neuralsde_1_18 = 'Neural SDE (2019)', 
    neuralsde_4_17 = 'Neural LNSDE (2024)',
)
model_mapping['ode-lstm'] = 'ODE-LSTM (2020)'
model_mapping['lddmm'] = 'LDDMM (2008)'
model_mapping['ts-lddmm'] = 'TS-LDDMM (ours)'

In [5]:
model_name_list = [
    'rnn', 'lstm', 'gru',
    'mtan', 'miam',
    'ode-lstm',
    "neuralsde_1_18", "neuralsde_4_17"
]

In [6]:
model_order = [
    "RNN (1999)",
    'LSTM (1997)',
    "GRU (2014)",
    'MTAN (2021)',
    'MIAM (2022)',
    'ODE-LSTM (2020)',
    'Neural SDE (2019)', 
    'Neural LNSDE (2024)',
    'LDDMM (2008)',
    'TS-LDDMM (ours)'
]

In [7]:
lst = []
for missing_rate in [0.0,0.3,0.5,0.7]:
    tlst = []
    for dataset in valid_datasets: 
        for model in model_name_list: 
            path = "results/{}/{}/{}_{}_{}_0".format(dataset,missing_rate,dataset,missing_rate,model)
            with open(path,"rb") as f:
                score = pickle.load(f)[-1]["f1score"]
                tlst.append([dataset,model,score])
    df = pd.DataFrame(tlst, columns=["dataset","model","score"])
    df["missing_rate"] = missing_rate
    lst.append(df)

for missing_rate in [0.0,0.3,0.5,0.7]: 
    df = pd.read_csv("results/lddmm_methods_{}_0.csv".format(missing_rate),index_col=0)
    df["missing_rate"] = missing_rate
    df["score"] = df["fscore"]
    df = df[["dataset", "model", "score", "missing_rate"]]
    lst.append(df)

df = pd.concat(lst)
df['model'] = df['model'].apply(lambda x : model_mapping[x])
df["score"] = df["score"].apply(lambda x : -x )

df = pd.pivot_table(df,"score",["missing_rate","dataset"],"model", aggfunc="first")
df = df.rank(axis=1,method="max").reset_index()
df = df.drop(["dataset"],axis=1)
df = df.groupby("missing_rate").mean()
df = df[model_order]
df = df.reset_index()
rank_df = pd.melt(df,id_vars="missing_rate")
rank_df = rank_df.set_index(["model","missing_rate"])

In [8]:
lst = []
for missing_rate in [0.0,0.3,0.5,0.7]:
    tlst = []
    for dataset in valid_datasets: 
        for model in model_name_list: 
            path = "results/{}/{}/{}_{}_{}_0".format(dataset,missing_rate,dataset,missing_rate,model)
            with open(path,"rb") as f:
                score = pickle.load(f)[-1]["f1score"]
                tlst.append([dataset,model,score])
    df = pd.DataFrame(tlst, columns=["dataset","model","score"])
    df["missing_rate"] = missing_rate
    lst.append(df)

for missing_rate in [0.0,0.3,0.5,0.7]: 
    df = pd.read_csv("results/lddmm_methods_{}_0.csv".format(missing_rate),index_col=0)
    df["missing_rate"] = missing_rate
    df["score"] = df["fscore"]
    df = df[["dataset", "model", "score", "missing_rate"]]
    lst.append(df)
        
df = pd.concat(lst)
df['model'] = df['model'].apply(lambda x : model_mapping[x])
mean_df = df.groupby(["model","missing_rate"]).score.mean()
std_df = df.groupby(["model","missing_rate"]).score.std()
df = pd.concat([mean_df,std_df,rank_df],axis=1)
df["mean_std"] = df.apply(lambda row : "${} \pm {}$  & {}".format(np.round(row[0],2),np.round(row[1],2),np.round(row[2],2)),axis=1)
df = df["mean_std"].reset_index()
df = pd.pivot_table(df,"mean_std","model","missing_rate",aggfunc="first")
df = df.loc[model_order]
print(df.style.to_latex(hrules = True, clines = "skip-last;data",multirow_align = "t"))

\begin{tabular}{lllll}
\toprule
missing_rate & 0.000000 & 0.300000 & 0.500000 & 0.700000 \\
model &  &  &  &  \\
\midrule
RNN (1999) & $0.64 \pm 0.21$  & 6.2 & $0.53 \pm 0.23$  & 6.6 & $0.48 \pm 0.21$  & 7.2 & $0.44 \pm 0.21$  & 6.07 \\
LSTM (1997) & $0.61 \pm 0.29$  & 6.0 & $0.57 \pm 0.29$  & 6.27 & $0.53 \pm 0.25$  & 6.07 & $0.51 \pm 0.29$  & 5.27 \\
GRU (2014) & $0.71 \pm 0.26$  & 4.2 & $0.68 \pm 0.28$  & 4.27 & $0.66 \pm 0.28$  & 3.73 & $0.59 \pm 0.28$  & 3.67 \\
MTAN (2021) & $0.59 \pm 0.28$  & 7.13 & $0.58 \pm 0.28$  & 5.8 & $0.54 \pm 0.29$  & 5.33 & $0.51 \pm 0.28$  & 5.0 \\
MIAM (2022) & $0.48 \pm 0.35$  & 6.93 & $0.42 \pm 0.33$  & 8.27 & $0.47 \pm 0.31$  & 6.93 & $0.35 \pm 0.31$  & 7.6 \\
ODE-LSTM (2020) & $0.63 \pm 0.24$  & 6.0 & $0.57 \pm 0.25$  & 6.53 & $0.51 \pm 0.24$  & 7.27 & $0.45 \pm 0.23$  & 6.73 \\
Neural SDE (2019) & $0.48 \pm 0.28$  & 7.67 & $0.47 \pm 0.26$  & 7.47 & $0.45 \pm 0.27$  & 7.13 & $0.45 \pm 0.25$  & 6.0 \\
Neural LNSDE (2024) & $0.7 \pm 0.27$  & 3.87 & 

<>:26: SyntaxWarning: invalid escape sequence '\p'
<>:26: SyntaxWarning: invalid escape sequence '\p'
/tmp/ipykernel_1943803/3397696954.py:26: SyntaxWarning: invalid escape sequence '\p'
  df["mean_std"] = df.apply(lambda row : "${} \pm {}$  & {}".format(np.round(row[0],2),np.round(row[1],2),np.round(row[2],2)),axis=1)
/tmp/ipykernel_1943803/3397696954.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["mean_std"] = df.apply(lambda row : "${} \pm {}$  & {}".format(np.round(row[0],2),np.round(row[1],2),np.round(row[2],2)),axis=1)


In [37]:
lst = []
for missing_rate in [0.0]:
    tlst = []
    for dataset in valid_datasets: 
        for model in model_name_list: 
            path = "results/{}/{}/{}_{}_{}_0".format(dataset,missing_rate,dataset,missing_rate,model)
            with open(path,"rb") as f:
                duration = pickle.load(f)[-1]["duration"]
                tlst.append([dataset,model,duration])
    df = pd.DataFrame(tlst, columns=["dataset","model","duration"])
    df["missing_rate"] = missing_rate
    lst.append(df)


for missing_rate in [0.0]: 
    df = pd.read_csv("results/lddmm_methods_{}_0.csv".format(missing_rate),index_col=0)
    df["missing_rate"] = missing_rate
    df["duration"] = df["time"]

    df = df[["dataset", "model", "duration", "missing_rate"]]
    lst.append(df)

#df = pd.concat(lst)
#df['model'] = df['model'].apply(lambda x : model_mapping[x])
#mean_df = df.groupby(["model","missing_rate"]).duration.mean()
#std_df = df.groupby(["model","missing_rate"]).duration.std()
#df = pd.concat([mean_df,std_df,rank_df],axis=1)
#df["mean_std"] = df.apply(lambda row : "${} \pm {}$  & {}".format(np.round(row[0],2),np.round(row[1],2),np.round(row[2],2)),axis=1)
#df = df["mean_std"].reset_index()
#df = pd.pivot_table(df,"mean_std","model","missing_rate",aggfunc="first")
#df = df.loc[model_order]
#print(df.style.to_latex(hrules = True, clines = "skip-last;data",multirow_align = "t"))

In [34]:
df[df.model.isin(["TS-LDDMM (ours)",'Neural LNSDE (2024)'])]

,dataset,model,duration,missing_rate
7,ArrowHead,Neural LNSDE (2024),90.944541,0.0
15,BME,Neural LNSDE (2024),74.872400,0.0
23,ECG200,Neural LNSDE (2024),59.338549,0.0
31,FacesUCR,Neural LNSDE (2024),544.192622,0.0
39,GunPoint,Neural LNSDE (2024),102.894789,0.0
47,PhalangesOutlinesCorrect,Neural LNSDE (2024),323.121866,0.0
55,Trace,Neural LNSDE (2024),126.800761,0.0
63,Cricket,Neural LNSDE (2024),538.478457,0.0
71,ERing,Neural LNSDE (2024),32.870731,0.0
79,Handwriting,Neural LNSDE (2024),297.825497,0.0


In [36]:
df.pivot_table("duration",columns="model",index="dataset",aggfunc="first")

model,GRU (2014),LDDMM (2008),LSTM (1997),MIAM (2022),MTAN (2021),Neural LNSDE (2024),Neural SDE (2019),ODE-LSTM (2020),RNN (1999),TS-LDDMM (ours)
dataset,,,,,,,,,,
ArrowHead,1.602968,222.533656,1.167632,7.962715,6.501672,90.944541,148.266738,83.896755,4.991670,213.180694
ArticularyWordRecognition,4.938596,858.754018,5.232652,60.331390,21.766998,87.396453,87.728271,90.068683,3.196109,843.711949
BME,0.640583,198.161349,0.770778,12.410461,7.270636,74.872400,64.717736,20.708190,1.166969,128.348649
Cricket,3.745402,5588.731756,8.196176,239.501436,87.112025,538.478457,472.715769,230.257229,5.046799,4795.217464
ECG200,0.636136,209.052101,0.717970,11.353386,2.931953,59.338549,24.000476,29.883200,0.831130,131.810511
ERing,1.664206,182.942168,0.700796,38.857554,4.011002,32.870731,12.629060,15.424053,1.077943,237.126642
FacesUCR,8.348208,1466.034100,5.400641,187.670161,69.142220,544.192622,111.371810,256.538715,5.676145,1493.942659
GunPoint,0.603658,140.980300,1.699749,9.379793,2.336470,102.894789,67.837670,44.308044,1.249933,145.708223
Handwriting,5.501382,1220.248428,8.814973,110.017528,19.085514,297.825497,85.792320,69.832008,5.305928,1166.545444


In [38]:
df

,dataset,model,duration,missing_rate
0,ArrowHead,lddmm,222.533656,0.0
1,BME,lddmm,198.161349,0.0
2,ECG200,lddmm,209.052101,0.0
3,FacesUCR,lddmm,1466.034100,0.0
4,GunPoint,lddmm,140.980300,0.0
5,PhalangesOutlinesCorrect,lddmm,2664.543673,0.0
6,Trace,lddmm,254.762670,0.0
7,Cricket,lddmm,5588.731756,0.0
8,ERing,lddmm,182.942168,0.0
9,Handwriting,lddmm,1220.248428,0.0
